# Predicción del precio de alquiler de la vivienda en la ciudad de Barcelona

Descripción del dataset:

|Columna|Descripción|Key|
|--|--|--|
|id|Identificador numérico de la vivienda||
|price|Precio de mercado de la vivienda||
|currency|Moneda|Euros / Mes|
|latitude|Latitud de las coordenadas geográficas de la vivienda||
|longitude|Longitud de las coordenadas geográficas de la vivienda||
|sq_meters|Metros cuadrados de la vivienda||
|sq_meters_built|Metros cuadrados construídos de la vivienda||
|rooms|Número de habitaciones||
|bathrooms|Número de baños||
|balcony|Indicador si la vivienda tiene balcón|1, 0|
|terrace|Indicador si la vivienda tiene terraza|1, 0|
|exterior|Indicador si la vivienda tiene una orientación exterior o interior en el edificio|1, 0|
|orientation|Orientación principal de la vivienda|norte, sur, este, oeste|
|floor|Piso de la vivienda||
|rooftop|Indicador si la vivienda es un ático|1, 0|
|elevator|Indicador si el edificio de la vivienda tiene ascensor|1, 0|
|doorman|Indicador si el edificio tiene portero|1,0|
|pool|Indicador si la vivienda cuenta con piscina o derecho de uso de piscina|1,0|
|ac|Indicador si tiene aire acondicionado|1,0|
|heating|Indicador si tiene calefacción|bomba, electric, gas, individual|
|year_built|Año de construcción||
|quality|Indicador de calidad de la vivienda|2 - En buen estado|
|city|Ciudad de la vivienda||
|neighborhood|Barrio de la vivienda||
|dist_city_center|Distancia en kilómetros al centro de la ciudad||
|furniture|Indicador si la vivienda cuenta con mobiliario|1: Sin Equipar; 2: Cocina Equipada; 3: Amueblado|
|garage|Indicador si la vivienda tiene garage|1, 0|
|property_type|Tipo de vivienda||
|garden|Indicador si la vivienda cuenta con jardín|1,0|
|closest_station|Nombre de la estación de metro más cercana||
|dist_closest_station|Distancia en kilómetros a la estación de metro más cercana||
|created_at|Fecha de creación del anuncio||
|last_seen|Fecha última en la que el anuncio fue publicado en la web||



### Exploratory Data Analysis

En éste punto queremos entender qué datos tenemos disponibles. Para ello realizaremos los siguientes puntos:

- Estadística descriptiva y calidad general de los datos: ¿Debemos hacer limpieza de los datos?, ¿Nos sirven todas las columnas?, ¿Nos sirven todas las instancias?, ¿Cómo debemos tratar los null values?
- Visualizaciones y análisis que ayuden a entender la distribución de las variables continuas y categorías independientemente: ¿Qué distribuciones siguen las variables continuas?, ¿Qué proporción de instancias tenemos para las variables categóricas?
- Visualizaciones y análisis que ayuden a entender la relación entre los atributos y la variable objetivo price: ¿Qué relación tienen las variables continuas con la variable objetivo price?, ¿Y las cariables categóricas?
- Alteración y creación de nuevas variables: ¿Podemos generar nuevas variables que se adapten más a nuestro objetivo que las que tenemos actualmente?
- Visualizaciones y análisis que ayuden a entender la posible correlación entre variables: ¿Cómo están relacionadas entre sí las variables interesantes para nuestro dominio?, ¿Debemos tener algún tipo de cuidado al respecto?

**Objetivos de éste punto:**

- Familiarizarnos con el dataset
- Generar un dataframe limpio, con las variables útiles para entrenar los modelos
- Entender qué posibles variables estén correlacionadas para evitar un sobreentrenamiento del modelo

#### Carga de librerías y dataset

In [1]:
# Library load
import os
import pandas as pd
import numpy as np

# Plotting library
import seaborn as sn
import matplotlib.pyplot as plt

# Funciones para hacer cálculo estadístico
import statsmodels.api as sm
from statsmodels.formula.api import ols

# sklearn packages
from sklearn.preprocessing import StandardScaler # Análisis de PCA
from sklearn import metrics # Calcula métricas para un modelo
from sklearn import tree # Cálculo de decision trees
from sklearn.tree import DecisionTreeClassifier # Generación de modelos de decision tree
from sklearn.ensemble import BaggingClassifier # Generación de modelos de bagging
from sklearn.ensemble import RandomForestClassifier # Generación de modelo de random forest
from sklearn.model_selection import train_test_split # Hace split entre training y testing
from sklearn.model_selection import cross_validate # trains model with cross validation
from sklearn.model_selection import GridSearchCV # Optimización de hiperparámetros para un modelo

# Confusion matrix viz
from mlxtend.evaluate import confusion_matrix # Calcula la matriz de confusion 
from mlxtend.plotting import plot_confusion_matrix #plot de la matriz de confusión

# Ignoring warning messaages
import warnings
warnings.filterwarnings('ignore') #ignora los errores en el notebook

In [2]:
raw_data = pd.read_csv('processed_renting_Barcelona (2).csv', delimiter = ',')
raw_data.head()

,id,price,currency,latitude,longitude,sq_meters,sq_meters_built,rooms,bathrooms,balcony,...,neighborhood,dist_city_center,furniture,garage,property_type,garden,closest_station,dist_closest_station,created_at,last_seen
0,536625,850,€/mes,41.401708,2.154077,52.0,55,2,1,NaN,...,Gràcia,2.026455,3.0,NaN,piso,NaN,Fontana,0.094111,9/1/2021 15:58,10/4/2021 6:01
1,545910,725,€/mes,41.407221,2.135569,32.0,37,2,1,1.0,...,Sant Gervasi - La Bonanova,3.582409,NaN,NaN,piso,NaN,Vallcarca,0.902561,9/2/2021 15:24,9/8/2021 12:29
2,570697,950,€/mes,41.411508,2.164608,NaN,72,3,1,NaN,...,El Baix Guinardó,2.663025,3.0,NaN,piso,NaN,Alfons X,0.188177,8/28/2021 23:52,8/28/2021 23:52
3,591588,750,€/mes,41.402256,2.140764,NaN,45,1,1,1.0,...,Sant Gervasi - La Bonanova,2.910067,3.0,NaN,piso,NaN,Lesseps,0.892917,8/29/2021 11:25,9/8/2021 12:29
4,610243,990,€/mes,41.405327,2.146929,NaN,45,1,1,NaN,...,Sarrià-Sant Gervasi,2.727149,3.0,NaN,piso,NaN,Lesseps,0.293784,8/30/2021 13:48,8/30/2021 14:04


#### Estadística descriptiva y calidad general de los datos

In [3]:
# Funciones útiles:
raw_data.describe() # Equivalente a función summary
raw_data.columns # Enumera las columnas del dataset
raw_data.isnull().sum() / len(raw_data) # Calcula la proporción de null values sobre todas las variables
raw_data.drop(['column_name'], axis=1) # Retira columnas que no nos sean útiles en el dataset
raw_data['column_name'].fillna(0, inplace = True) # Sustituye los Null values de una columna por 0. Podemos cambiar el 0 por cualquier otro valor

KeyError: "['column_name'] not found in axis"

#### Visualizaciones y análisis que ayuden a entender la distribución de las variables continuas y categorías independientemente

In [ ]:
# Histogramas para entender la distribución de las variables
raw_data.hist(bins=20, figsize=(25, 20))
# Para variables categóricas podemos hacer la función groupby
raw_data[['column_name', 'price']].groupby('column_name').agg(
    # Number of instances per category
    category_count=('price', "count"),
    # Mean price
    mean_price=('price', "mean"),
    # Median price
    median_price=('price', "median"),
    # Min price
    min_price=('price', min),
    # Max price
    max_price=('price', max),
    # Standard deviation
    stantard_deviation=('price', "std"))

#### Visualizaciones y análisis que ayuden a entender la relación entre las variables categóricas y la variable objetivo price

In [ ]:
# Test ANOVA para 
model = ols('price ~ column_name', data=raw_data).fit()
aov_table = sm.stats.anova_lm(model, typ=2)

# Scatter matrix, similar a la función pairs. Cuidado que solo sive para variables numéricas
pd.plotting.scatter_matrix(raw_data, alpha=0.2, figsize=(20, 20), diagonal='kde')

# Análisis de PCA. Cuidado que solo sive para variables numéricas y que hay que retirar la variable objetivo
StandardScaler().fit_transform(raw_data)

#### Alteración y creación de nuevas variables

In [ ]:
dataset_numerico =  raw_data[['price', 'sq_meters_built', 'dist_city_center']]
dataset_numerico

#### Visualizaciones y análisis que ayuden a entender la posible correlación entre variables

In [ ]:
# Análisis de correlación
corrMatrix = raw_data.corr()
# Visualización de la matriz de correlación
sn.heatmap(corrMatrix, annot=True)
plt.show()

### Modelo de Bagging

En éste punto crearemos nuestro primer modelo de bagging. Para ello realizaremos los siguientes puntos:

- Selección de la métrica de optimización: ¿Qué métrica debemos utilizar para optimizar y valorar la calidad de nuestros modelos?
- Split del dataset en training y test: ¿Cómo debemos separar los datos para entrenar y validar nuestro modelo?
- Optimización de hiperparámetros mediante Cross Validation: ¿Qué hiperparámetros deberíamos utilizar para entrenar nuestro modelo?
- Entrenamiento del modelo y análisis de los resultados: ¿Es bueno nuestro modelo?

**Objetivos de éste punto:**

- Definir cómo mediremos los resultados de nuestro modelo
- Generar y entrenar un modelo de datos

#### Selección de la métrica de optimización

- Tipos de scoring metrics: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

#### Split del dataset en training y test

In [ ]:
X = dataset_numerico.drop(['price'], axis = 1)
y = dataset_numerico[['price']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

#### Optimización de hiperparámetros mediante Cross Validation

- Decision tree classifier: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
- Bagging classifier: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html

In [ ]:
# we enumerate the values to try
parameters = [{"max_depth":[2,3,4,5,6,7,9,10,12,15], "min_samples_split":[2,5,10]}]

#instantiate the classifier
decision_tree_model = DecisionTreeClassifier()

# Grid search function
grid_bag = GridSearchCV(cv = 10, estimator=decision_tree_model, param_grid=parameters, scoring="accuracy")
grid_bag.fit(X_train, y_train)
grid_bag.best_estimator_

In [ ]:
# we enumerate the values to try
parameters = [{"n_estimators":[1,5,10,20,50,100,200]]

#instantiate the classifier
bagging_model = BaggingClassifier(decision_tree_model)

# Grid search function
grid_bag = GridSearchCV(cv = 10, estimator=bagging_model, param_grid=parameters, scoring="accuracy")
grid_bag.fit(X_train, y_train)
grid_bag.best_estimator_

#### Entrenamiento del modelo y análisis de los resultados

In [ ]:
# Creamos el modelo con los hiperpametros seleccionados en el punto anterior
bag = BaggingClassifier(DecisionTreeClassifier(max_depth = n, min_samples_split = m), n_estimators = n)
# Entrenamos el modelo con el dataset de entrenamiento mediante cross validation
model = cross_validate(bag, X_train, y_train, xv = 10, scoring="accuracy")
# Generación de las predicciones para el dataset de testing
y_pred = model.predict(X_test)
#error rate
error = 1.0 - metrics.accuracy_score(y_test, y_pred)
# Plot de la matriz de confusión
fig, ax = plot_confusion_matrix(conf_mat=confusion_matrix(y_test, y_pred, binary = False))
plt.show()

# Ejemplo de modelo de Bagging

In [ ]:
raw_data.head()

Retiramos las variables que no nos importan para el modelo

In [ ]:
rent_dataset = raw_data[['price', 'sq_meters_built', 'rooms', 'bathrooms', 'balcony', 'terrace', 'exterior', 'orientation', 'floor', 'rooftop', 'elevator', 'pool', 'ac', 'heating', 'year_built', 'neighborhood', 'dist_city_center', 'property_type', 'dist_closest_station']]
rent_dataset.head()

Podemos asumir que para algunas de las variables, los valores nulos son en realidad 0:

In [ ]:
features_to_null = ['balcony', 'terrace', 'exterior', 'rooftop', 'elevator', 'pool', 'ac']
for i in range(0,len(features_to_null)):
   rent_dataset[features_to_null[i]].fillna(0, inplace = True) 
rent_dataset.head()

Para otras, como por ejemplo la property_age, podemos transformarla a una variable categórica incluyendo los valores nulos como una categoría de unknown:

In [ ]:
rent_dataset['property_age'] = 2022 - rent_dataset['year_built']
rent_dataset.head()

In [ ]:
# Scatter matrix, similar a la función pairs. Cuidado que solo sive para variables numéricas
rent_dataset[['property_age']].hist(bins=60, figsize=(25, 20))

Apreciamos pequeños "clusters de viviendas, los cuales podemos agrupar en categorías según la edad de la vivienda:

In [ ]:
def conditions(x):
    if x < 0:
        return "Unknown"
    elif ((x >= 0) & (x < 40)):
        return "0 - 40"
    elif ((x >= 40) & (x < 70)):
        return "40 - 70"
    elif ((x >= 70) & (x < 120)):
        return "70 - 120"
    elif ((x >= 120) & (x < 150)):
        return "120 - 150"
    elif ((x >= 150)):
        return "+150"
    else:
        return "Unknown"

func = np.vectorize(conditions)
rent_dataset['property_age_cat'] = func(rent_dataset['property_age'])
rent_dataset.head()

Del dataset que tenemos, generamos un subset con valores numéricos únicamente:

In [ ]:
cat_features = ['neighborhood', 'property_type', 'orientation', 'heating', 'property_age_cat']
rent_dataset_numeric = rent_dataset.drop(cat_features, axis = 1)
rent_dataset_numeric

Comprobamos que no hay nulls

In [ ]:
rent_dataset_numeric_no_nulls = rent_dataset_numeric.drop(['year_built', 'property_age', 'floor'], axis = 1)
rent_dataset_numeric_no_nulls.isnull().sum() / len(rent_dataset_numeric_no_nulls)

Generamos el split

In [ ]:
X = rent_dataset_numeric_no_nulls.drop(['price'], axis = 1)
y = rent_dataset_numeric_no_nulls[['price']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

Ejemplos de scoring parameter: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

En este caso optamos por un R2.

Hacemos grid search con cross validation para encontrar los parámetros óptimos de nuestro modelo

In [ ]:
# we enumerate the values to try
parameters = [{"max_depth":[2,3,4,5,6,7,9], "min_samples_split":[2,5,10]}]

#instantiate the classifier
decision_tree_model = DecisionTreeClassifier()

# Grid search function
grid_bag = GridSearchCV(cv = 10, estimator=decision_tree_model, param_grid=parameters, scoring="r2")
grid_bag.fit(X_train, y_train)

In [ ]:
grid_bag.best_params_

In [ ]:
# we enumerate the values to try
parameters = {"n_estimators":[1,5,10,20,50,100,200]}

#instantiate the classifier
decision_tree_model = DecisionTreeClassifier(max_depth = 9, min_samples_split = 10)

#instantiate the classifier
bagging_model = BaggingClassifier(decision_tree_model)

# Grid search function
grid_bag = GridSearchCV(cv = 10, estimator=bagging_model, param_grid=parameters, scoring="r2")
grid_bag.fit(X_train, y_train)
grid_bag.best_params_

A continuación, con los parámetros óptimos del modelo, procedemos a hacer un xvalidation para entender la performance del modelo:

In [ ]:
# Creamos el modelo con los hiperpametros seleccionados en el punto anterior
bag = BaggingClassifier(DecisionTreeClassifier(max_depth = 9, min_samples_split = 10), n_estimators = 20)
# Entrenamos el modelo con el dataset de entrenamiento mediante cross validation
model = cross_validate(bag, X_train, y_train, cv = 10, scoring = "r2")

In [ ]:
for i, score in enumerate(model["test_score"]):
    print(f"Accuracy for the fold no. {i} on the test set: {score}")

Nos encontramos con que hay mucha variación con los datos de entrenamiento (R2 de 0.35 o de 0.74). Ésto nos indica que hay mucha variación en los datos y que probablemente tengamos outlayers o grupos de instancias que no estamos prediciendo correctamente

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('R2 del modelo de Bagging')
ax1.boxplot(model["test_score"])

Generamos las predicciones y calculamos el resultado de validación:

In [ ]:
bag = BaggingClassifier(DecisionTreeClassifier(max_depth = 9, min_samples_split = 10), n_estimators = 20)
bag.fit(X_train, y_train)

Error del modelo:

In [ ]:
y_pred = bag.predict(X_train)
#error rate
metrics.r2_score(y_train, y_pred)

Error en validación

In [ ]:
y_pred = bag.predict(X_test)
#error rate
metrics.r2_score(y_test, y_pred)